In [ ]:
#Importing Modules
import cv2
import numpy as np
from os import listdir
from os.path import isfile, join

In [ ]:
#Face Features Database
face_detector = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")

In [ ]:
#Extract faces from the Photos
def face_extract(img):
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    face = face_detector.detectMultiScale(gray, 1.3, 5)
    
    if face is not ():
        [x, y, w, h] = face[0]
        face_image = gray[x:x+w , y:y+h]
        return face_image, face[0]
    
    return None, None

In [ ]:
#Create your own Dataset at Runtime
def dataset_maker(dataset_size = 100, dataset_location = "./faces/ayush/"):
    cap = cv2.VideoCapture(0)
    count = 0
    while True:
        ret, frame = cap.read()
        face, _ = face_extract(frame)
        if face is not None:
            face = cv2.resize(face, (300,300))
            file_name = "./faces/ayush/image" + str(count) + ".jpg"
            cv2.imwrite(file_name, face)
            
            cv2.putText(face, str(count), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
            cv2.imshow("Dataset Preparation", face)
            count = count+1
            if cv2.waitKey(1) & 0xFF == ord('q') or count >= dataset_size:
                break
                
    cap.release()
    cv2.destroyAllWindows()
    print("Dataset made sucessefully")

In [ ]:
#Train a model to Recognise the person or obkect in provided dataset.
def model_training(dataset_location = "./faces/ayush/"):
    onlyfiles = [file for file in listdir(dataset_location) if isfile(join(dataset_location, file))]
    Training_Data, Labels = [], []
    
    for i, files in enumerate(onlyfiles):
        image_path = dataset_location + onlyfiles[i]
        images = cv2.imread(image_path, 0) # 0 means cv2.IMREAD_GRAYSCALE
        Training_Data.append(np.asarray(images, dtype=np.uint8))
        Labels.append(i)

    Labels = np.asarray(Labels, dtype=np.int32)
    model = cv2.face_LBPHFaceRecognizer.create()

    model.train(np.asarray(Training_Data), np.asarray(Labels))
    print("Model trained sucessefully")
    return model

In [ ]:
#Predict the possibility of the detected person in the detected
def predict(model):
    cap = cv2.VideoCapture(0)
    count, label, confidence = 0, 0, 0
    while True:
        ret, img = cap.read()
        if ret == True:
            face, coordinates = face_extract(img)
            if coordinates is not None:
                [x, y, w, h] = coordinates
                cv2.resize(face, (300,300))
                cv2.rectangle(img, (x,y), (x+h, y+w), (0,255,0), 2)
                
                if count == 0:
                    label, confidence = model.predict(face)
                    cv2.putText(img, "Label: "+str(label), (50,50), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                    cv2.putText(img, "Confidence: "+str(confidence), (50,80), cv2.FONT_HERSHEY_COMPLEX, 1, (0,255,0), 2)
                
                count = count%60
            cv2.imshow("Window", img)
            if cv2.waitKey(1) & 0xFF == ord('q'):
                break
                
    cv2.destroyAllWindows()
    cap.release()

In [ ]:
dataset_maker(100)
model = model_training()
predict(model)